In [1]:
library(tidyverse)
library(corrplot)
library(cluster)
library(caTools)
library(class)
library(Matrix)
library(Rtsne)
library(PerformanceAnalytics)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
corrplot 0.95 loaded


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack


Loading required package: xts

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



######################### Warning from 'xts' package ##########################
#                                                                             #
# T

In [2]:
sparse_RP <- function(scdata, p, seedn) {
  # for random projection; note scdata: m*n, m is the feature dimensions, n is the
  # sample number; p is the reduced dimension
  m = nrow(scdata)  #the number of features
  n = ncol(scdata)  #number of samples/cells
  # set.seed(123)#a flag to fix the random number
  s = sqrt(m)  #according to the paper 'Very Sparse Random Projection'
  # x0 = sample(c(sqrt(s),0, -sqrt(s)), size= m*p, replace=TRUE, prob=c(1/(2*s), 1
  # - 1/s, 1/(2*s)))
  if (seedn%%1 == 0) {
    # integer
    set.seed(seedn)
    x0 = sample(c(sqrt(s), 0, -sqrt(s)), size = m * p, replace = TRUE, prob = c(1/(2 * 
                                                                                     s), 1 - 1/s, 1/(2 * s)))
  } else {
    x0 = sample(c(sqrt(s), 0, -sqrt(s)), size = m * p, replace = TRUE, prob = c(1/(2 * 
                                                                                     s), 1 - 1/s, 1/(2 * s)))
  }
  # return(x)
  x <- Matrix(x0, nrow = m, byrow = TRUE, sparse = TRUE)  #reshape a vector to a sparse matrix
  return(x)  #the same format, feature*sample
}

gaussian_RP <- function(scdata, p, seedn) {
  # Function to perform Gaussian Random Projection
  # scdata: m*n, where m is the number of features and n is the number of samples
  # p: target dimension for projection
  # seedn: seed for reproducibility
  
  m <- nrow(scdata)  # Number of features (original dimension)
  n <- ncol(scdata)  # Number of samples/cells
  
  # Set seed for reproducibility
  if (seedn%%1 == 0) {  # Check if seedn is integer
    set.seed(seedn)
  }
  
  # Generate Gaussian random projection matrix
  # Entries are drawn from N(0, 1/p)
  proj_matrix <- matrix(rnorm(m * p, mean = 0, sd = 1 / sqrt(p)), nrow = m, ncol = p)
  
  # Perform the random projection
  # Resulting projected data: feature-reduced dimension * n (samples)
#   projected_data <- as.matrix(scdata) %*% proj_matrix
  
  return(proj_matrix)
}

SSSE_RP <- function(scdata, p, seedn = 123) {
  # Function for SSSE Random Projection
  # scdata: m*n matrix (m: features, n: samples)
  # p: reduced dimension
  # seedn: seed for reproducibility
  
  # Dimensions of the input data
  m <- nrow(scdata)  # Number of features
  n <- ncol(scdata)  # Number of samples
  
  # Set seed for reproducibility
  set.seed(seedn)
  
  # Generate sparse random projection matrix
  # x0: a sequence of 1 to p, repeated to cover all features
  x0 <- rep(1:p, ceiling(m / p))  
  S <- sample(x0, size = m, replace = FALSE)  # Sample row indices without replacement
  
  # Randomly assign +1 or -1 values to the projection matrix
  x1 <- sample(c(1, -1), size = m, replace = TRUE, prob = c(1/2, 1/2))  
  
  # Construct a sparse random projection matrix (p * m)
  RP_mtx <- sparseMatrix(i = S, j = 1:m, x = x1, dims = c(p, m))
  
  # Convert the input data to a sparse matrix format
#   scmat <- Matrix(as.matrix(scdata), sparse = TRUE)
  
  # Perform the projection: R (p*m) %*% scdata (m*n) = projmat (p*n)
#   projmat <- RP_mtx %*% scmat
  
  # Return the projection matrix and the projected data
  return(RP_mtx)
}

In [3]:
data_path = "/mnt/nrdstor/wanlab/xinchaowu/lung_cancer_subtype/data/"
tpm_file = "binary_classification_tpm.csv"

tpm = read.csv(paste0(data_path, tpm_file), header = T)
# discard the first column
tpm = tpm[, -1]
# tpm = t(tpm)

# tpm_inuse = tpm[1:5000,]
# n_tpm = data.matrix(tpm_inuse)
n_tpm = data.matrix(tpm)
duplicates <- duplicated(n_tpm)
print(sum(duplicates))  # Number of duplicate rows
n_tpm_unique <- n_tpm[!duplicates, ]

# check for constant or zero-variance features
# apply(n_tpm, 2, var)
n_tpm_filtered <- n_tpm_unique[, apply(n_tpm_unique, 2, var) > 0]
# pca_result <- prcomp(n_tpm_filtered, scale. = TRUE, center = TRUE, rank. = 50)

duplicates <- duplicated(n_tpm_filtered)
print(sum(duplicates))  # Number of duplicate rows
n_tpm_filtered <- n_tpm_filtered[!duplicates, ]

n_tpm_filtered <- n_tpm_filtered[, apply(n_tpm_filtered, 2, var) > 0]

# Orignal data
n_tpm = data.matrix(n_tpm_filtered)
or_tpm <- t(scale(t(n_tpm)))
or_d = as.dist(1 - cor(t(or_tpm)))
or_dm = data.matrix(or_d)
or_dm_v = or_dm[lower.tri(or_dm)]
or_dm_v = data.matrix(or_dm_v)

# TSNE on original data
tsne_out = Rtsne(
  n_tpm_filtered,
  dims = 3,
  pca = T,
  verbose = F
)
tsne_tpm <- tsne_out$Y
tsnen_tpm <- scale(t(tsne_tpm))
tsne_d = as.dist(1 - cor(tsnen_tpm))
tsne_dm = data.matrix(tsne_d)
tsne_dm_v = tsne_dm[lower.tri(tsne_dm)]
tsne_dm_v = data.matrix(tsne_dm_v)

# UMAP on original data
library(umap) 
umap_out = umap(
    n_tpm_filtered, 
    n_components = 3, 
    random_state = 42
) 
umap_tpm = umap_out[["layout"]]
umapn_tpm <- scale(t(umap_tpm))
umap_d = as.dist(1 - cor(umapn_tpm))
umap_dm = data.matrix(umap_d)
umap_dm_v = umap_dm[lower.tri(umap_dm)]
umap_dm_v = data.matrix(umap_dm_v)

scale_0_1 <- function(x) {
  (x - min(x)) / (max(x) - min(x))
}

or_dm_v_scaled <- scale_0_1(or_dm_v)
tsne_dm_v_scaled <- scale_0_1(tsne_dm_v)
umap_dm_v_scaled <- scale_0_1(umap_dm_v)

[1] 1
[1] 0


In [4]:
# Chart
chart.Correlation.nostars <- function (R, histogram = TRUE, method = c("pearson", "kendall", 
                                          "spearman"), ...) 
{
    x = checkData(R, method = "matrix")
    if (missing(method)) 
        method = method[1]
    
    panel.cor <- function(x, y, digits = 2, prefix = "", 
                        use = "pairwise.complete.obs", method = "pearson", 
                        cex.cor, ...) {
        usr <- par("usr")
        on.exit(par(usr))
        par(usr = c(0, 1, 0, 1))
        r <- cor(x, y, use = use, method = method)
        txt <- format(c(r, 0.123456789), digits = digits)[1]
        txt <- paste(prefix, txt, sep = "")
        if (missing(cex.cor)) 
          cex <- 0.8/strwidth(txt)
        test <- cor.test(as.numeric(x), as.numeric(y), method = method)
        # Signif <- symnum(test$p.value, corr = FALSE, na = FALSE, 
        #                  cutpoints = c(0, 0.001, 0.01, 0.05, 0.1, 1), symbols = c("***", 
        #                                                                           "**", "*", ".", " "))
        text(0.5, 0.5, txt, cex = 3.5)#cex * (abs(r) + 0.3)/1.3)
        # text(0.8, 0.8, Signif, cex = cex, col = 2)
      }
    
    f <- function(t) {
    dnorm(t, mean = mean(x), sd = sd.xts(x))
    }
    
    hist.panel = function(x, ... = NULL) {
    par(new = TRUE)
    hist(x, col = "light gray", probability = TRUE, 
         axes = FALSE, main = "", breaks = "FD")
    lines(density(x, na.rm = TRUE), col = "red", lwd = 1)
    rug(x)
    }
    
    dotargs <- list(...)
    dotargs$method <- NULL
    rm(method)
    
    if (histogram) 
    # panel.smooth is defined in PerformanceAnalytics package
        pairs(x, gap = 0, lower.panel = panel.smooth, upper.panel = panel.cor, 
              diag.panel = hist.panel)
    else pairs(x, gap = 0, 
      lower.panel = function(x, y, ...) {panel.smooth(
            x, y, col = "lightblue", pch=16, lwd=2, cex=0.5, col.smooth = "red",
              )
          },
      # panel.smooth, 
      upper.panel = panel.cor,
      cex.axis = 2,
      cex.labels = 3, ...)
}

In [5]:
# N = c(20, 80, 140, 200, 260, 320, 380, 440, 500)
N = c(20, 100, 200, 400)
output_path = "/mnt/nrdstor/wanlab/xinchaowu/lung_cancer_subtype/binary_output/cor_plot/"

for (n in N){
    print(n)
    # sparse random projection
    rM = sparse_RP(t(n_tpm_filtered),n,42)
    rp_tpm = 1/sqrt(n) * t(rM) %*% t(n_tpm)
    rpn_tpm <- scale(rp_tpm)
    rp_d = as.dist(1 - cor(rpn_tpm))
    rp_dm = data.matrix(rp_d)
    rp_dm_v = rp_dm[lower.tri(rp_dm)]
    rp_dm_v = data.matrix(rp_dm_v)
    rp_dm_v_scaled = scale_0_1(rp_dm_v)

    # gaussian random projection
    gM = gaussian_RP(t(n_tpm_filtered),n,42)
    gp_tpm = 1/sqrt(n) * t(gM) %*% t(n_tpm)
    gpn_tpm <- t(scale(t(gp_tpm)))
    gp_d = as.dist(1 - cor(gpn_tpm))
    gp_dm = data.matrix(gp_d)
    gp_dm_v = gp_dm[lower.tri(gp_dm)]
    gp_dm_v = data.matrix(gp_dm_v)
    gp_dm_v_scaled = scale_0_1(gp_dm_v)

    # SSSE random projection
    ssse_out = SSSE_RP(t(n_tpm_filtered),n,42)
    ssse_tpm = 1/sqrt(n) * ssse_out %*% t(n_tpm)
    ssse_n_tpm <- scale(ssse_tpm)
    ssse_d = as.dist(1 - cor(ssse_n_tpm))
    ssse_dm = data.matrix(ssse_d)
    ssse_dm_v = ssse_dm[lower.tri(ssse_dm)]
    ssse_dm_v = data.matrix(ssse_dm_v)
    ssse_dm_v_scaled = scale_0_1(ssse_dm_v)

    # PCA
    pca <- prcomp(n_tpm_filtered, scale. = T, center = T, rank = n)
    pca_tpm <- pca$x
    pcan_tpm <- scale(t(pca_tpm))
    pca_d = as.dist(1 - cor(pcan_tpm))
    pca_dm = data.matrix(pca_d)
    pca_dm_v = pca_dm[lower.tri(pca_dm)]
    pca_dm_v = data.matrix(pca_dm_v)
    pca_dm_v_scaled = scale_0_1(pca_dm_v)
    
    # cor_df = data.frame(Ori.= or_dm_v, 
    #             Sp.RP = rp_dm_v, Gau.RP = gp_dm_v, S3E.RP = ssse_dm_v,
    #             PCA = pca_dm_v, tSNE = tsne_dm_v, UMAP = umap_dm_v)
    cor_df = data.frame(Ori.= or_dm_v,
                PCA = pca_dm_v, tSNE = tsne_dm_v, UMAP = umap_dm_v,
                RP = gp_dm_v, Sp.RP = rp_dm_v, S3E.RP = ssse_dm_v)

    tiff(paste(output_path, n,"_RP.tiff",sep=""),width = 15,height = 15,units = "cm",res = 600)
    # print(chart.Correlation.nostars(cor_df[c("Ori.","PCA","tSNE","UMAP", "Gau.RP", "Sp.RP", "S3E.RP")],
                # histogram = FALSE ,pch=21))
    print(chart.Correlation.nostars(cor_df[c("Ori.","PCA","tSNE","UMAP", "RP")],
                histogram = FALSE ,pch=21))
    dev.off()
}

[1] 20


Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”


NULL
[1] 100


Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”


NULL
[1] 200


Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”


NULL
[1] 400


Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”
Warning message in par(usr):
“argument 1 does not name a graphical parameter”


NULL


In [ ]:
pca <- prcomp(n_tpm_filtered, scale. = T, center = T, rank = n)
pca_tpm <- pca$x
pcan_tpm <- scale(t(pca_tpm))
pca_d = as.dist(1 - cor(pcan_tpm))
pca_dm = data.matrix(pca_d)
pca_dm_v = pca_dm[lower.tri(pca_dm)]
pca_dm_v = data.matrix(pca_dm_v)


par(cex = 4)
cor_df_test = data.frame(Original= or_dm_v, 
            PCA = pca_dm_v, tSNE = tsne_dm_v, UMAP = umap_dm_v
            )

tiff(paste(output_path, "test.tiff",sep=""),width = 15,height = 15,units = "cm",res = 300)
chart.Correlation.nostars(cor_df_test[c("Original","PCA", "tSNE", "UMAP")],
            histogram = FALSE ,pch=21)
dev.off()